In [55]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException 
import re
import sys
import os
import dotenv 
dotenv.load_dotenv()
from datetime import datetime

key = os.getenv("token")

In [56]:
##sacar las skid
parametro = 'ovd'
url = "https://sky-scrapper.p.rapidapi.com/api/v1/flights/searchAirport"

querystring = {"query":f"{parametro}"}

headers = {
	"x-rapidapi-key": key,
	"x-rapidapi-host": "sky-scrapper.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'status': True, 'timestamp': 1729436244608, 'data': [{'skyId': 'OVD', 'entityId': '95565102', 'presentation': {'title': 'Asturias', 'suggestionTitle': 'Asturias (OVD)', 'subtitle': 'Spain'}, 'navigation': {'entityId': '95565102', 'entityType': 'AIRPORT', 'localizedName': 'Asturias', 'relevantFlightParams': {'skyId': 'OVD', 'entityId': '95565102', 'flightPlaceType': 'AIRPORT', 'localizedName': 'Asturias'}, 'relevantHotelParams': {'entityId': '46362301', 'entityType': 'CITY', 'localizedName': 'Santiago del Monte'}}}]}


In [57]:
#  skyId': 'MAD', 'entityId': '95565077'
# 'skyId': 'AGP', 'entityId': '95565095'
# 'skyId': 'OVD', 'entityId': '95565102'

In [58]:


url = "https://sky-scrapper.p.rapidapi.com/api/v2/flights/searchFlightsComplete"

querystring = {"originSkyId":"MAD","destinationSkyId":"OVD","originEntityId":"95565077","destinationEntityId":"95565102","date":"2024-11-01","returnDate":"2024-11-03","adults":"2","currency":"EUR"}

headers = {
	"x-rapidapi-key": key,
	"x-rapidapi-host": "sky-scrapper.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

if response.status_code == 200:
    data = response.json()

{'status': True, 'message': 'success', 'data': {'context': {'status': 'complete', 'sessionId': 'ClQIARJQCk4KJDkyZWM5OTc3LWEzMmMtNGJkOC1iNGIyLWMyMjc0ZGJhNDdjMRABGiRiNmY1NTMwYS00ZDI1LTQ2ZTktOWJmOS1hZDlhYWIwODIwNGYSKHVzc18wNmE1NjkwMC0xZTEzLTQ0YWUtYTA2OS02ODhmMWQ3ZjY0ZGI=', 'totalResults': 49}, 'itineraries': [{'id': '13870-2411011640--32222-0-15140-2411011755|15140-2411031740--32222-0-13870-2411031855', 'price': {'raw': 610.93, 'formatted': '611 €', 'pricingOptionId': 'T5dC2a4YYLb7'}, 'legs': [{'id': '13870-2411011640--32222-0-15140-2411011755', 'origin': {'id': 'MAD', 'entityId': '95565077', 'name': 'Madrid', 'displayCode': 'MAD', 'city': 'Madrid', 'country': 'Spain', 'isHighlighted': False}, 'destination': {'id': 'OVD', 'entityId': '95565102', 'name': 'Asturias', 'displayCode': 'OVD', 'city': 'Santiago del Monte', 'country': 'Spain', 'isHighlighted': False}, 'durationInMinutes': 75, 'stopCount': 0, 'isSmallestStops': False, 'departure': '2024-11-01T16:40:00', 'arrival': '2024-11-01T17:5

In [59]:
resp= data["data"]
precio= resp["itineraries"][0]["price"]["raw"]

origen= resp["itineraries"][0]["legs"][0]["origin"]["city"]

destino= resp["itineraries"][0]["legs"][0]["destination"]["city"]

duracion_minuts= resp["itineraries"][0]["legs"][0]["durationInMinutes"]

salida= resp["itineraries"][0]["legs"][0]["departure"]
llegada= resp["itineraries"][0]["legs"][0]["arrival"]

aerolinea= resp["itineraries"][0]["legs"][0]["carriers"]["marketing"][0]["name"]

In [60]:
dicct= {
       "ciudad_origen":origen,
       "ciudad_destino":destino,
       "salida":salida,
       "llegada":llegada,
       "duracion(min)":duracion_minuts,
       "aerolinea":aerolinea,
       "precio":precio
        }
dicct

{'ciudad_origen': 'Madrid',
 'ciudad_destino': 'Santiago del Monte',
 'salida': '2024-11-01T16:40:00',
 'llegada': '2024-11-01T17:55:00',
 'duracion(min)': 75,
 'aerolinea': 'Iberia',
 'precio': 610.93}

In [61]:
lista_dicct=[]

for vuelo in range(0,len(resp["itineraries"])):
        resp= data["data"]
        precio= resp["itineraries"][0]["price"]["raw"]
        for i in range(0,2):
                origen= resp["itineraries"][vuelo]["legs"][i]["origin"]["city"]
                destino= resp["itineraries"][vuelo]["legs"][i]["destination"]["city"]

                duracion_minuts= resp["itineraries"][vuelo]["legs"][i]["durationInMinutes"]

                salida= resp["itineraries"][vuelo]["legs"][i]["departure"]
                llegada= resp["itineraries"][vuelo]["legs"][i]["arrival"]

                aerolinea= resp["itineraries"][vuelo]["legs"][i]["carriers"]["marketing"][0]["name"]

                dicct= {
                "ciudad_origen":origen,
                "ciudad_destino":destino,
                "salida":salida,
                "llegada":llegada,
                "duracion(min)":duracion_minuts,
                "aerolinea":aerolinea,
                "precio":precio
                }
                lista_dicct.append(dicct)
df_vuelos_Asturias= pd.DataFrame(lista_dicct)
df_vuelos_Asturias.head(2)

,ciudad_origen,ciudad_destino,salida,llegada,duracion(min),aerolinea,precio
0,Madrid,Santiago del Monte,2024-11-01T16:40:00,2024-11-01T17:55:00,75,Iberia,610.93
1,Santiago del Monte,Madrid,2024-11-03T17:40:00,2024-11-03T18:55:00,75,Iberia,610.93


In [62]:
df_vuelos_Asturias.isnull().sum()



ciudad_origen     0
ciudad_destino    0
salida            0
llegada           0
duracion(min)     0
aerolinea         0
precio            0
dtype: int64

In [63]:
df_vuelos_Asturias.to_csv('../Datos/vuelos_Asturias.csv')

In [64]:
df_vuelos = pd.read_csv('../Datos/vuelos_Asturias.csv')
df_vuelos.head(2)

,Unnamed: 0,ciudad_origen,ciudad_destino,salida,llegada,duracion(min),aerolinea,precio
0,0,Madrid,Santiago del Monte,2024-11-01T16:40:00,2024-11-01T17:55:00,75,Iberia,610.93
1,1,Santiago del Monte,Madrid,2024-11-03T17:40:00,2024-11-03T18:55:00,75,Iberia,610.93


In [65]:
df_vuelos.dtypes

Unnamed: 0          int64
ciudad_origen      object
ciudad_destino     object
salida             object
llegada            object
duracion(min)       int64
aerolinea          object
precio            float64
dtype: object

In [66]:
df_vuelos.drop(columns=['Unnamed: 0'], inplace=True)

In [67]:
df_vuelos[['fecha_salida', 'hora_salida']] = df_vuelos['salida'].str.split('T', expand=True)
df_vuelos[['fecha_llegada', 'hora_llegada']] = df_vuelos['llegada'].str.split('T', expand=True)
df_vuelos.dtypes

ciudad_origen      object
ciudad_destino     object
salida             object
llegada            object
duracion(min)       int64
aerolinea          object
precio            float64
fecha_salida       object
hora_salida        object
fecha_llegada      object
hora_llegada       object
dtype: object

In [68]:
df_vuelos['fecha_salida'] = pd.to_datetime(df_vuelos['fecha_salida'])
df_vuelos['hora_salida'] = pd.to_datetime(df_vuelos['hora_salida'])
df_vuelos['fecha_llegada'] = pd.to_datetime(df_vuelos['fecha_llegada'])
df_vuelos['hora_llegada'] = pd.to_datetime(df_vuelos['hora_llegada'])


C:\Users\HP\AppData\Local\Temp\ipykernel_10332\979191819.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_vuelos['hora_salida'] = pd.to_datetime(df_vuelos['hora_salida'])
C:\Users\HP\AppData\Local\Temp\ipykernel_10332\979191819.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_vuelos['hora_llegada'] = pd.to_datetime(df_vuelos['hora_llegada'])


In [69]:
df_vuelos.drop(columns=['salida', 'llegada'], inplace=True)

In [70]:
df_vuelos.head()

,ciudad_origen,ciudad_destino,duracion(min),aerolinea,precio,fecha_salida,hora_salida,fecha_llegada,hora_llegada
0,Madrid,Santiago del Monte,75,Iberia,610.93,2024-11-01,2024-10-20 16:40:00,2024-11-01,2024-10-20 17:55:00
1,Santiago del Monte,Madrid,75,Iberia,610.93,2024-11-03,2024-10-20 17:40:00,2024-11-03,2024-10-20 18:55:00
2,Madrid,Santiago del Monte,75,Iberia,610.93,2024-11-01,2024-10-20 12:50:00,2024-11-01,2024-10-20 14:05:00
3,Santiago del Monte,Madrid,75,Iberia,610.93,2024-11-03,2024-10-20 17:40:00,2024-11-03,2024-10-20 18:55:00
4,Madrid,Santiago del Monte,75,Iberia,610.93,2024-11-01,2024-10-20 07:25:00,2024-11-01,2024-10-20 08:40:00


In [71]:
df_vuelos.to_csv('../Datos/vuelos_Asturias.csv')